In [1]:
# ============ 1. НАСТРОЙКА ОКРУЖЕНИЯ ============
import nest_asyncio
nest_asyncio.apply()

import asyncio
import aiohttp
import json
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Dict, Optional, Any, Union
import logging
import hashlib
import re
import os
import warnings
warnings.filterwarnings('ignore')

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print(" Библиотеки загружены и окружение настроено!")

# ============ 2. МОДЕЛИ PYDANTIC ============
from pydantic import BaseModel, Field, validator
from enum import Enum
from sentence_transformers import SentenceTransformer

class EventCategory(str, Enum):
    """15+ категорий событий"""
    CONCERT = "concert"
    THEATER = "theater"
    EXHIBITION = "exhibition"
    FESTIVAL = "festival"
    EDUCATION = "education"
    PARTY = "party"
    SPORT = "sport"
    QUEST = "quest"
    EXCURSION = "excursion"
    SHOW = "show"
    STANDUP = "standup"
    KIDS = "kids"
    FASHION = "fashion"
    GASTRONOMY = "gastronomy"
    CINEMA = "cinema"
    LECTURE = "lecture"
    MASTERCLASS = "masterclass"
    TOUR = "tour"
    OTHER = "other"

class PlaceModel(BaseModel):
    """Полная модель места"""
    id: Optional[int] = None
    title: str = Field(..., description="Название места")
    address: Optional[str] = None
    subway: Optional[str] = None
    coords: Optional[Dict[str, float]] = None
    phone: Optional[str] = None
    site_url: Optional[str] = None
    city: Optional[str] = None
    is_closed: bool = False
    working_hours: Optional[Dict] = None
    
    @property
    def full_address(self) -> str:
        parts = []
        if self.title:
            parts.append(self.title)
        if self.address:
            parts.append(self.address)
        if self.subway:
            parts.append(f"м. {self.subway}")
        return ", ".join(parts)

class DateModel(BaseModel):
    """Модель даты события"""
    start: int  # timestamp
    end: Optional[int] = None
    is_continuous: bool = False
    
    @property
    def start_dt(self) -> datetime:
        return datetime.fromtimestamp(self.start)
    
    @property
    def end_dt(self) -> Optional[datetime]:
        return datetime.fromtimestamp(self.end) if self.end else None
    
    @property
    def formatted(self) -> str:
        start_str = self.start_dt.strftime("%d.%m.%Y %H:%M")
        if self.end:
            end_str = self.end_dt.strftime("%d.%m.%Y %H:%M")
            if start_str[:10] == end_str[:10]:
                return f"{start_str} - {end_str[11:]}"
            return f"{start_str} - {end_str}"
        return start_str

class PriceModel(BaseModel):
    """Модель цены"""
    is_free: bool = False
    min: Optional[float] = None
    max: Optional[float] = None
    currency: str = "RUB"
    description: Optional[str] = None
    
    @property
    def display(self) -> str:
        if self.is_free:
            return "Бесплатно"
        elif self.min and self.max:
            return f"{self.min:.0f} - {self.max:.0f} ₽"
        elif self.min:
            return f"от {self.min:.0f} ₽"
        return self.description or "Цена не указана"

class ImageModel(BaseModel):
    """Модель изображения"""
    url: str
    thumbnail: Optional[str] = None
    source: Optional[Dict] = None

class TagModel(BaseModel):
    """Модель тега"""
    name: str
    slug: Optional[str] = None
    id: Optional[int] = None

class ParticipantModel(BaseModel):
    """Модель участника"""
    name: str
    role: Optional[str] = None
    images: Optional[List[Dict]] = None

class EventModel(BaseModel):
    """ПОЛНАЯ МОДЕЛЬ СОБЫТИЯ"""
    id: int
    title: str
    short_title: Optional[str] = None
    description: Optional[str] = None
    slug: Optional[str] = None
    category: EventCategory
    tags: List[TagModel] = []
    dates: List[DateModel] = []
    publication_date: Optional[int] = None
    age_restriction: Optional[str] = None
    place: Optional[PlaceModel] = None
    place_id: Optional[int] = None
    price: Optional[PriceModel] = None
    images: List[ImageModel] = []
    video: Optional[Dict] = None
    participants: List[ParticipantModel] = []
    url: str
    site_url: Optional[str] = None
    buy_url: Optional[str] = None
    favorites_count: int = 0
    comments_count: int = 0
    external_id: Optional[str] = None
    location: str = "msk"
    is_free: bool = False
    is_exclusive: bool = False
    is_editors_choice: bool = False
    is_approved: bool = True
    parsed_at: datetime = Field(default_factory=datetime.now)
    
    @property
    def embedding_text(self) -> str:
        parts = [self.title]
        if self.description:
            parts.append(self.description[:500])
        if self.tags:
            parts.extend([tag.name for tag in self.tags[:5]])
        if self.place:
            if self.place.title:
                parts.append(self.place.title)
            if self.place.address:
                parts.append(self.place.address)
        parts.append(self.category.value)
        return " ".join(parts)
    
    @property
    def date_range_text(self) -> str:
        if not self.dates:
            return "Дата не указана"
        if len(self.dates) == 1:
            return self.dates[0].formatted
        dates_str = [date.formatted for date in self.dates[:3]]
        if len(self.dates) > 3:
            dates_str.append(f"... еще {len(self.dates)-3}")
        return "; ".join(dates_str)
    
    class Config:
        json_encoders = {
            datetime: lambda dt: dt.isoformat(),
        }

print(" Pydantic модели созданы!")

# ============ 3. БАЗА ДАННЫХ SQLite С ВЕКТОРНЫМИ ЭМБЕДДИНГАМИ ============
class EventVectorDatabaseSQLite:
    """Векторная база данных на основе SQLite с эмбеддингами"""
    
    def __init__(self, db_path: str = "events_vector.db", embedding_model: str = "all-MiniLM-L6-v2"):
        self.db_path = db_path
        try:
            # Отключаем логирование для ускорения
            import logging
            logging.getLogger("sentence_transformers").setLevel(logging.WARNING)
            logging.getLogger("transformers").setLevel(logging.WARNING)
            
            self.model = SentenceTransformer(embedding_model)
            self.embedding_dim = self.model.get_sentence_embedding_dimension()
        except Exception as e:
            logger.error(f"Ошибка загрузки модели: {e}")
            logger.info("Использую заглушку для тестирования...")
            # Создаем заглушку для тестирования
            self.embedding_dim = 384
            self.model = None
        self.conn = None
        self._init_db()
        print(f" Векторная БД SQLite создана: {db_path}")
        print(f" Размерность эмбеддингов: {self.embedding_dim}")
    
    def _init_db(self):
        """Инициализация базы данных"""
        self.conn = sqlite3.connect(self.db_path)
        cursor = self.conn.cursor()
        
        # Основная таблица событий
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            description TEXT,
            category TEXT,
            dates_text TEXT,
            price_text TEXT,
            place_text TEXT,
            url TEXT,
            city TEXT,
            city_name TEXT,
            parsed_at TEXT,
            source TEXT DEFAULT 'kudago',
            image_count INTEGER DEFAULT 0,
            embedding_text TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Таблица для векторных эмбеддингов
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS event_embeddings (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            event_id INTEGER NOT NULL,
            embedding BLOB NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (event_id) REFERENCES events (id)
        )
        ''')
        
        # Таблица для дополнительных данных
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS event_details (
            id INTEGER PRIMARY KEY,
            event_id INTEGER,
            dates_json TEXT,
            price_json TEXT,
            place_json TEXT,
            tags_json TEXT,
            images_json TEXT,
            participants_json TEXT,
            age_restriction TEXT,
            FOREIGN KEY (event_id) REFERENCES events (id)
        )
        ''')
        
        # Индексы для быстрого поиска
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_events_city ON events(city)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_events_category ON events(category)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_events_date ON events(parsed_at)')
        
        self.conn.commit()
    
    def _blob_to_array(self, blob):
        """Конвертация BLOB в numpy array"""
        return np.frombuffer(blob, dtype=np.float32)
    
    def _array_to_blob(self, array):
        """Конвертация numpy array в BLOB"""
        return array.astype(np.float32).tobytes()
    
    def create_embedding(self, text: str):
        """Создание векторного эмбеддинга для текста"""
        try:
            if not text or not text.strip():
                # Возвращаем нулевой вектор если текст пустой
                return np.zeros(self.embedding_dim, dtype=np.float32)
            
            if self.model is None:
                # Заглушка для тестирования - случайные эмбеддинги
                logger.debug("Использую заглушку для эмбеддингов")
                return np.random.randn(self.embedding_dim).astype(np.float32)
            
            # Создаем эмбеддинг без прогресс-бара
            embedding = self.model.encode(text, show_progress_bar=False)
            
            # Если embedding уже numpy array, конвертируем в нужный формат
            if isinstance(embedding, np.ndarray):
                return embedding.astype(np.float32)
            else:
                return np.array(embedding, dtype=np.float32)
                
        except Exception as e:
            logger.error(f"Ошибка создания эмбеддинга: {e}")
            return np.zeros(self.embedding_dim, dtype=np.float32)
    
    def save_event_with_embedding(self, event_data: Dict) -> bool:
        """Сохранение события с векторным эмбеддингом"""
        try:
            event_id = event_data.get('id')
            if not event_id:
                logger.warning(f"Событие без ID: {event_data.get('title', 'Без названия')}")
                return False
            
            cursor = self.conn.cursor()
            
            # Проверяем существование события
            cursor.execute("SELECT id FROM events WHERE id = ?", (event_id,))
            exists = cursor.fetchone()
            
            # Подготавливаем текст для эмбеддинга
            embedding_text = ""
            parts = []
            
            if event_data.get('title'):
                parts.append(str(event_data['title']))
            if event_data.get('description'):
                parts.append(str(event_data['description'])[:500])
            if event_data.get('tags'):
                tags = event_data['tags']
                if isinstance(tags, list):
                    tag_names = []
                    for tag in tags[:5]:
                        if isinstance(tag, dict):
                            tag_names.append(tag.get('name', ''))
                        elif isinstance(tag, str):
                            tag_names.append(tag)
                    if tag_names:
                        parts.append(' '.join(tag_names))
            if event_data.get('category'):
                parts.append(str(event_data['category']))
            
            embedding_text = " ".join(parts)
            
            # Основные поля
            title = str(event_data.get('title', ''))[:200]
            description = str(event_data.get('description', ''))[:1000]
            category = str(event_data.get('category', ''))[:50]
            
            # Даты
            dates_text_list = event_data.get('dates_text', [])
            if isinstance(dates_text_list, list):
                dates_text = ','.join([str(d) for d in dates_text_list])[:500]
            else:
                dates_text = str(dates_text_list)[:500]
            
            price_text = str(event_data.get('price_text', ''))[:200]
            place_text = str(event_data.get('place_text', ''))[:500]
            url = str(event_data.get('url', ''))[:500]
            city = str(event_data.get('city', ''))[:10]
            city_name = str(event_data.get('city_name', ''))[:50]
            parsed_at = str(event_data.get('parsed_at', datetime.now().isoformat()))
            image_count = event_data.get('image_count', 0)
            
            if exists:
                # Обновляем существующее событие
                cursor.execute('''
                UPDATE events SET
                    title=?, description=?, category=?, dates_text=?,
                    price_text=?, place_text=?, url=?, city=?,
                    city_name=?, parsed_at=?, image_count=?, embedding_text=?
                WHERE id=?
                ''', (
                    title, description, category, dates_text,
                    price_text, place_text, url, city,
                    city_name, parsed_at, image_count, embedding_text,
                    event_id
                ))
            else:
                # Вставляем новое событие
                cursor.execute('''
                INSERT INTO events (
                    id, title, description, category, dates_text,
                    price_text, place_text, url, city,
                    city_name, parsed_at, image_count, embedding_text
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    event_id, title, description, category, dates_text,
                    price_text, place_text, url, city,
                    city_name, parsed_at, image_count, embedding_text
                ))
            
            # Создаем и сохраняем эмбеддинг
            embedding = self.create_embedding(embedding_text)
            embedding_blob = self._array_to_blob(embedding)
            
            # Сохраняем эмбеддинг
            cursor.execute('''
            INSERT OR REPLACE INTO event_embeddings (event_id, embedding)
            VALUES (?, ?)
            ''', (event_id, embedding_blob))
            
            # Сохраняем детали
            self._save_event_details(event_id, event_data)
            
            self.conn.commit()
            return True
            
        except Exception as e:
            logger.error(f"Ошибка сохранения события {event_data.get('id')}: {e}")
            if self.conn:
                self.conn.rollback()
            return False
    
    def _save_event_details(self, event_id: int, event_data: Dict):
        """Сохранение дополнительных данных события"""
        try:
            cursor = self.conn.cursor()
            
            def safe_to_json(data, default=None):
                try:
                    if data is None:
                        return '{}'
                    return json.dumps(data, ensure_ascii=False)
                except:
                    return json.dumps(default or {}, ensure_ascii=False)
            
            dates_json = safe_to_json(event_data.get('dates'))
            price_json = safe_to_json(event_data.get('price'))
            place_json = safe_to_json(event_data.get('place'))
            tags_json = safe_to_json(event_data.get('tags'), [])
            images_json = safe_to_json(event_data.get('images'), [])
            participants_json = safe_to_json(event_data.get('participants'), [])
            age_restriction = str(event_data.get('age_restriction', ''))[:50]
            
            cursor.execute("SELECT id FROM event_details WHERE event_id = ?", (event_id,))
            exists = cursor.fetchone()
            
            if exists:
                cursor.execute('''
                UPDATE event_details SET
                    dates_json=?, price_json=?, place_json=?, tags_json=?,
                    images_json=?, participants_json=?, age_restriction=?
                WHERE event_id=?
                ''', (
                    dates_json, price_json, place_json, tags_json,
                    images_json, participants_json, age_restriction,
                    event_id
                ))
            else:
                cursor.execute('''
                INSERT INTO event_details (
                    event_id, dates_json, price_json, place_json, tags_json,
                    images_json, participants_json, age_restriction
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    event_id, dates_json, price_json, place_json, tags_json,
                    images_json, participants_json, age_restriction
                ))
                
        except Exception as e:
            logger.error(f"Ошибка сохранения деталей {event_id}: {e}")
    
    def save_events_batch(self, events: List[Dict]) -> Dict[str, int]:
        """Пакетное сохранение событий"""
        success = 0
        failed = 0
        
        print("  Сохранение событий в базу данных...")
        for event in events:
            if self.save_event_with_embedding(event):
                success += 1
            else:
                failed += 1
        
        return {"success": success, "failed": failed, "total": len(events)}
    
    def search_similar(self, query: str, n_results: int = 10, city: str = None, category: str = None):
        """Поиск похожих событий по векторному сходству"""
        try:
            # Создаем эмбеддинг для запроса
            query_embedding = self.create_embedding(query)
            
            cursor = self.conn.cursor()
            
            # Получаем все эмбеддинги
            if city and category:
                cursor.execute('''
                SELECT e.id, e.title, e.category, e.city, ed.embedding
                FROM events e
                JOIN event_embeddings ed ON e.id = ed.event_id
                WHERE e.city = ? AND e.category = ?
                ''', (city, category))
            elif city:
                cursor.execute('''
                SELECT e.id, e.title, e.category, e.city, ed.embedding
                FROM events e
                JOIN event_embeddings ed ON e.id = ed.event_id
                WHERE e.city = ?
                ''', (city,))
            elif category:
                cursor.execute('''
                SELECT e.id, e.title, e.category, e.city, ed.embedding
                FROM events e
                JOIN event_embeddings ed ON e.id = ed.event_id
                WHERE e.category = ?
                ''', (category,))
            else:
                cursor.execute('''
                SELECT e.id, e.title, e.category, e.city, ed.embedding
                FROM events e
                JOIN event_embeddings ed ON e.id = ed.event_id
                ''')
            
            rows = cursor.fetchall()
            
            # Вычисляем косинусное сходство
            results = []
            for event_id, title, category, city, embedding_blob in rows:
                # Конвертируем BLOB в numpy array
                event_embedding = self._blob_to_array(embedding_blob)
                
                # Вычисляем косинусное сходство
                similarity = np.dot(query_embedding, event_embedding) / (
                    np.linalg.norm(query_embedding) * np.linalg.norm(event_embedding) + 1e-10
                )
                
                results.append({
                    'event_id': event_id,
                    'title': title,
                    'category': category,
                    'city': city,
                    'similarity': float(similarity)
                })
            
            # Сортируем по сходству
            results.sort(key=lambda x: x['similarity'], reverse=True)
            
            # Берем топ-N результатов
            top_results = results[:n_results]
            
            # Получаем полную информацию о событиях
            for result in top_results:
                cursor.execute('''
                SELECT description, dates_text, price_text, place_text, url
                FROM events WHERE id = ?
                ''', (result['event_id'],))
                
                event_row = cursor.fetchone()
                if event_row:
                    result.update({
                        'description': event_row[0],
                        'dates_text': event_row[1],
                        'price_text': event_row[2],
                        'place_text': event_row[3],
                        'url': event_row[4]
                    })
            
            return top_results
            
        except Exception as e:
            logger.error(f"Ошибка поиска: {e}")
            return []
    
    def get_stats(self) -> Dict:
        """Получение статистики"""
        try:
            cursor = self.conn.cursor()
            
            cursor.execute("SELECT COUNT(*) FROM events")
            total_events = cursor.fetchone()[0] or 0
            
            cursor.execute("SELECT COUNT(*) FROM event_embeddings")
            total_embeddings = cursor.fetchone()[0] or 0
            
            cursor.execute("SELECT COUNT(DISTINCT city) FROM events")
            cities_count = cursor.fetchone()[0] or 0
            
            cursor.execute("SELECT COUNT(DISTINCT category) FROM events")
            categories_count = cursor.fetchone()[0] or 0
            
            cursor.execute("SELECT city, COUNT(*) FROM events GROUP BY city")
            by_city = {row[0]: row[1] for row in cursor.fetchall()}
            
            return {
                "total_events": total_events,
                "total_embeddings": total_embeddings,
                "cities_count": cities_count,
                "categories_count": categories_count,
                "events_by_city": by_city,
                "db_path": self.db_path
            }
        except Exception as e:
            logger.error(f"Ошибка получения статистики: {e}")
            return {}
    
    def close(self):
        """Закрытие соединения"""
        if self.conn:
            self.conn.close()

print(" Векторная БД SQLite создана!")

# ============ 4. УЛУЧШЕННЫЙ ПАРСЕР KUDAGO ============
class EnhancedKudaGoParser:
    """Улучшенный парсер с поддержкой категорий"""
    
    def __init__(self, city: str = "msk"):
        self.base_url = "https://kudago.com/public-api/v1.4"
        self.city = city
        self.city_name = "Москва" if city == "msk" else "Санкт-Петербург"
        
        # Проверенные категории для Москвы (основные, которые точно работают)
        self.working_categories_msk = [
            ("concert", "Концерты"),
            ("theater", "Театр"), 
            ("exhibition", "Выставки"),
            ("festival", "Фестивали"),
            ("education", "Образование"),
            ("party", "Вечеринки"),
            ("quest", "Квесты"),
            ("kids", "Детские"),
            ("fashion", "Мода"),
            ("cinema", "Кино"),
        ]
        
        # Проверенные категории для Санкт-Петербурга
        self.working_categories_spb = [
            ("concert", "Концерты"),
            ("theater", "Театр"), 
            ("exhibition", "Выставки"),
            ("festival", "Фестивали"),
            ("education", "Образование"),
            ("party", "Вечеринки"),
            ("kids", "Детские"),
            ("cinema", "Кино"),
        ]
        
        # Все возможные категории для тестирования
        self.all_categories = [
            ("concert", "Концерты"), ("theater", "Театр"), ("exhibition", "Выставки"),
            ("festival", "Фестивали"), ("education", "Образование"), ("party", "Вечеринки"),
            ("sport", "Спорт"), ("quest", "Квесты"), ("excursion", "Экскурсии"),
            ("show", "Шоу"), ("standup", "Стендап"), ("kids", "Детские"),
            ("fashion", "Мода"), ("gastronomy", "Гастрономия"), ("cinema", "Кино"),
        ]
        
        # Используем рабочие категории в зависимости от города
        if city == "msk":
            self.categories = self.working_categories_msk
        else:
            self.categories = self.working_categories_spb
            
        self.max_events_per_category = 30  # Уменьшаем для тестирования
        self.page_size = 20  # Уменьшаем размер страницы
    
    async def fetch_page(self, session, category: str, page: int) -> Optional[Dict]:
        """Загрузка одной страницы событий с улучшенными параметрами"""
        try:
            # Более простой запрос с минимальными параметрами
            params = {
                'categories': category,
                'location': self.city,
                'page_size': self.page_size,
                'page': page,
                'fields': 'id,title,description,dates,place,price,images,tags,age_restriction',
                'text_format': 'text'
            }
            
            async with session.get(f"{self.base_url}/events/", params=params, 
                                   timeout=aiohttp.ClientTimeout(total=30)) as response:
                if response.status == 200:
                    return await response.json()
                elif response.status == 400:
                    # Для категории 400, пробуем без location
                    logger.debug(f"Пробуем запрос без location для категории {category}")
                    params.pop('location', None)
                    
                    async with session.get(f"{self.base_url}/events/", params=params, 
                                           timeout=aiohttp.ClientTimeout(total=30)) as response2:
                        if response2.status == 200:
                            return await response2.json()
                        else:
                            logger.debug(f"Категория {category} недоступна (статус {response2.status})")
                            return None
                elif response.status == 429:
                    await asyncio.sleep(2)  # Увеличиваем паузу при ограничении
                    return None
                else:
                    logger.debug(f"HTTP Error {response.status} для категории {category}")
                    return None
        except asyncio.TimeoutError:
            logger.warning(f"Таймаут для категории {category}")
            return None
        except Exception as e:
            logger.warning(f"Ошибка загрузки категории {category}: {e}")
            return None
    
    async def enrich_place_info(self, session, place_data: Dict) -> Dict:
        """Обогащение информации о месте"""
        if not place_data or not isinstance(place_data, dict):
            return {}
        place_id = place_data.get('id')
        if not place_id:
            return place_data
        try:
            # Простой запрос информации о месте
            async with session.get(f"{self.base_url}/places/{place_id}/", 
                                   timeout=aiohttp.ClientTimeout(total=10)) as response:
                if response.status == 200:
                    detailed = await response.json()
                    # Обновляем только основные поля
                    updated_fields = {}
                    for field in ['title', 'address', 'subway', 'coords', 'phone', 'site_url', 'city']:
                        if field in detailed:
                            updated_fields[field] = detailed[field]
                    place_data.update(updated_fields)
        except Exception as e:
            # Игнорируем ошибки обогащения места
            pass
        return place_data
    
    def parse_raw_event(self, raw: Dict, category_name: str) -> Optional[Dict]:
        """Парсинг сырого события в словарь с обработкой ошибок"""
        try:
            event_id = raw.get('id')
            title = raw.get('title')
            
            if not event_id or not title:
                return None
            
            # Базовые данные события
            event_data = {
                'id': event_id,
                'title': str(title),
                'description': str(raw.get('description', ''))[:500],
                'category': category_name,
                'url': raw.get('site_url', f"https://kudago.com/{self.city}/event/{event_id}/"),
                'is_free': raw.get('is_free', False),
                'age_restriction': raw.get('age_restriction'),
                'favorites_count': raw.get('favorites_count', 0),
                'comments_count': raw.get('comments_count', 0),
            }
            
            # Даты
            dates = []
            dates_text = []
            for date_info in raw.get('dates', []):
                if isinstance(date_info, dict):
                    start = date_info.get('start')
                    if start:
                        try:
                            start_dt = datetime.fromtimestamp(start)
                            dates.append({
                                'start': start, 
                                'end': date_info.get('end'),
                                'is_continuous': date_info.get('is_continuous', False)
                            })
                            text = start_dt.strftime("%d.%m.%Y %H:%M")
                            end = date_info.get('end')
                            if end:
                                try:
                                    end_dt = datetime.fromtimestamp(end)
                                    if start_dt.date() == end_dt.date():
                                        text += f" - {end_dt.strftime('%H:%M')}"
                                    else:
                                        text += f" - {end_dt.strftime('%d.%m.%Y %H:%M')}"
                                except:
                                    pass
                            dates_text.append(text)
                        except:
                            continue
            
            event_data['dates'] = dates
            event_data['dates_text'] = dates_text
            
            # Место
            raw_place = raw.get('place', {})
            event_data['place'] = raw_place
            if isinstance(raw_place, dict) and 'id' in raw_place:
                event_data['place_id'] = raw_place['id']
            
            # Текст места
            place_parts = []
            if isinstance(raw_place, dict):
                if raw_place.get('title'):
                    place_parts.append(str(raw_place['title']))
                if raw_place.get('address'):
                    place_parts.append(str(raw_place['address']))
            event_data['place_text'] = ", ".join(place_parts)[:300]
            
            # Цена
            raw_price = raw.get('price', {})
            if isinstance(raw_price, dict):
                event_data['price'] = {
                    'is_free': raw_price.get('is_free', False), 
                    'min': raw_price.get('min'),
                    'max': raw_price.get('max'), 
                    'currency': raw_price.get('currency', 'RUB'),
                    'description': raw_price.get('description')
                }
                
                # Текст цены
                price_text = ""
                if raw_price.get('is_free'):
                    price_text = "Бесплатно"
                elif raw_price.get('min') and raw_price.get('max'):
                    price_text = f"{raw_price['min']} - {raw_price['max']} ₽"
                elif raw_price.get('min'):
                    price_text = f"от {raw_price['min']} ₽"
                elif raw_price.get('description'):
                    price_text = str(raw_price['description'])[:100]
                else:
                    price_text = "Цена не указана"
                    
                event_data['price_text'] = price_text
            else:
                event_data['price_text'] = "Цена не указана"
            
            # Изображения
            images = []
            for img in raw.get('images', [])[:3]:  # Ограничиваем 3 изображения
                if isinstance(img, dict):
                    images.append({
                        'url': str(img.get('image', '')), 
                        'thumbnail': str(img.get('thumbnail', '')), 
                        'source': img.get('source', {})
                    })
            event_data['images'] = images
            event_data['image_count'] = len(images)
            
            # Теги
            tags = []
            for tag in raw.get('tags', [])[:5]:  # Ограничиваем 5 тегов
                if isinstance(tag, dict):
                    tags.append({
                        'name': str(tag.get('name', '')), 
                        'slug': tag.get('slug'), 
                        'id': tag.get('id')
                    })
                elif isinstance(tag, str):
                    tags.append({'name': str(tag)})
            event_data['tags'] = tags
            
            # Город
            event_data['city'] = self.city
            event_data['city_name'] = self.city_name
            event_data['parsed_at'] = datetime.now().isoformat()
            
            return event_data
        except Exception as e:
            logger.error(f"Ошибка парсинга события {raw.get('id')}: {e}")
            return None
    
    async def parse_category(self, category_code: str, category_name: str) -> List[Dict]:
        """Парсинг всей категории с пагинацией"""
        all_events = []
        page = 1
        max_pages = 3  # Ограничиваем 3 страницами для тестирования
        
        print(f"   Категория: {category_name}")
        
        async with aiohttp.ClientSession() as session:
            while len(all_events) < self.max_events_per_category and page <= max_pages:
                data = await self.fetch_page(session, category_code, page)
                
                if not data or 'results' not in data:
                    break
                
                events = data['results']
                if not events:
                    break
                
                # Парсим события
                batch_events = []
                for raw_event in events:
                    event_dict = self.parse_raw_event(raw_event, category_name)
                    if event_dict:
                        batch_events.append(event_dict)
                
                # Пропускаем обогащение мест для скорости (опционально)
                enriched_events = batch_events
                # Если нужно обогащение:
                # enriched_events = []
                # for event_dict in batch_events:
                #     if event_dict.get('place'):
                #         event_dict['place'] = await self.enrich_place_info(session, event_dict['place'])
                #     enriched_events.append(event_dict)
                
                all_events.extend(enriched_events)
                
                print(f"    Страница {page}: {len(events)} событий, спаршено {len(enriched_events)}")
                
                if not data.get('next'):
                    break
                
                page += 1
                await asyncio.sleep(1)  # Пауза между запросами
        
        print(f"     Всего: {len(all_events)} событий")
        return all_events
    
    async def parse_all_categories(self, max_concurrent: int = 2) -> Dict[str, List[Dict]]:
        """Парсинг всех категорий параллельно"""
        print(f" Начинаем парсинг города: {self.city_name}")
        print(f" Категорий: {len(self.categories)}")
        print(f" Цель: {self.max_events_per_category} событий на категорию")
        print("-" * 50)
        
        # Создаем задачи для каждой категории
        tasks = []
        for cat_code, cat_name in self.categories:
            task = self.parse_category(cat_code, cat_name)
            tasks.append((cat_name, task))
        
        # Ограничиваем одновременные запросы
        semaphore = asyncio.Semaphore(max_concurrent)
        
        async def run_with_limit(cat_name, task):
            async with semaphore:
                try:
                    return cat_name, await task
                except Exception as e:
                    logger.error(f"Ошибка при парсинге категории {cat_name}: {e}")
                    return cat_name, []
        
        # Запускаем все задачи
        all_tasks = [run_with_limit(cat_name, task) for cat_name, task in tasks]
        results_list = await asyncio.gather(*all_tasks, return_exceptions=True)
        
        # Обрабатываем результаты
        results = {}
        total_events = 0
        
        for result in results_list:
            if isinstance(result, Exception):
                logger.error(f"Исключение при парсинге: {result}")
                continue
            
            cat_name, events = result
            results[cat_name] = events
            total_events += len(events)
        
        print("\n" + "="*50)
        print(f" ПАРСИНГ ЗАВЕРШЕН!")
        print(f"  Город: {self.city_name}")
        print(f" Всего событий: {total_events}")
        print("="*50)
        
        # Выводим статистику по категориям
        for cat_name, events in results.items():
            if events:
                print(f"  {cat_name}: {len(events)} событий")
            else:
                print(f"  {cat_name}: 0 событий (нет данных)")
        
        return results

print(" Улучшенный парсер создан!")

# ============ 5. ОСНОВНОЙ ПАЙПЛАЙН ============
class DataPipeline:
    """Полный пайплайн обработки данных"""
    
    def __init__(self, city: str = "msk"):
        self.city = city
        self.parser = EnhancedKudaGoParser(city)
        self.vector_db = EventVectorDatabaseSQLite(db_path=f"events_{city}_vector.db")
        self.events_dict = None
        self.pydantic_events = None
    
    async def run_full_pipeline(self):
        """Запуск полного пайплайна"""
        print("="*60)
        print(" ЗАПУСК ПОЛНОГО ПАЙПЛАЙНА")
        print("="*60)
        
        # 1. Парсинг данных
        print("\n1. ПАРСИНГ ДАННЫХ")
        print("-"*40)
        self.events_dict = await self.parser.parse_all_categories(max_concurrent=2)
        
        # Объединяем все события
        all_events_raw = []
        for category, events in self.events_dict.items():
            all_events_raw.extend(events)
        
        print(f" Всего спаршено: {len(all_events_raw)} событий")
        
        if not all_events_raw:
            print(" Нет данных для обработки")
            return None
        
        # 2. Конвертация в Pydantic модели
        print("\n2. КОНВЕРТАЦИЯ В PYDANTIC")
        print("-"*40)
        self.pydantic_events = self.convert_to_pydantic(all_events_raw)
        print(f" Конвертировано: {len(self.pydantic_events)} моделей")
        
        # 3. Сохранение в JSON
        print("\n3. СОХРАНЕНИЕ В JSON")
        print("-"*40)
        json_path = self.save_to_json(all_events_raw)
        
        # 4. Загрузка в векторную БД SQLite
        print("\n4. ЗАГРУЗКА В ВЕКТОРНУЮ БД SQLite")
        print("-"*40)
        save_result = self.vector_db.save_events_batch(all_events_raw)
        print(f" Сохранено: {save_result['success']} событий")
        print(f"  Ошибок: {save_result['failed']}")
        
        # 5. Статистика
        print("\n5. СТАТИСТИКА")
        print("-"*40)
        self.show_statistics(all_events_raw)
        
        # 6. Пример поиска
        print("\n6. ПРИМЕР ПОИСКА")
        print("-"*40)
        if all_events_raw:
            # Пример поиска
            search_query = "концерт музыка"
            print(f" Поиск: '{search_query}'")
            similar_events = self.vector_db.search_similar(search_query, n_results=3, city=self.city)
            
            if similar_events:
                print(f" Найдено {len(similar_events)} похожих событий:")
                for i, event in enumerate(similar_events, 1):
                    print(f"{i}. {event['title'][:50]}...")
                    print(f"   Категория: {event['category']}")
                    print(f"   Сходство: {event['similarity']:.3f}")
                    if event.get('price_text'):
                        print(f"   Цена: {event['price_text']}")
                    print()
            else:
                print(" Похожих событий не найдено")
        
        print("\n" + "="*60)
        print(" ПАЙПЛАЙН УСПЕШНО ЗАВЕРШЕН!")
        print("="*60)
        
        return {
            'total_events': len(all_events_raw),
            'pydantic_count': len(self.pydantic_events),
            'json_file': json_path,
            'db_stats': self.vector_db.get_stats(),
            'city': self.city
        }
    
    def convert_to_pydantic(self, events_raw: List[Dict]) -> List[EventModel]:
        """Конвертация словарей в Pydantic модели"""
        pydantic_events = []
        converted = 0
        errors = 0
        
        for event_dict in events_raw[:100]:  # Ограничиваем для скорости
            try:
                category_str = event_dict.get('category', '').lower()
                try:
                    # Пробуем найти подходящую категорию
                    if category_str == "концерты":
                        category = EventCategory.CONCERT
                    elif category_str == "театр":
                        category = EventCategory.THEATER
                    elif category_str == "выставки":
                        category = EventCategory.EXHIBITION
                    elif category_str == "фестивали":
                        category = EventCategory.FESTIVAL
                    elif category_str == "образование":
                        category = EventCategory.EDUCATION
                    elif category_str == "вечеринки":
                        category = EventCategory.PARTY
                    elif category_str == "спорт":
                        category = EventCategory.SPORT
                    elif category_str == "квесты":
                        category = EventCategory.QUEST
                    elif category_str == "экскурсии":
                        category = EventCategory.EXCURSION
                    elif category_str == "шоу":
                        category = EventCategory.SHOW
                    elif category_str == "стендап":
                        category = EventCategory.STANDUP
                    elif category_str == "детские":
                        category = EventCategory.KIDS
                    elif category_str == "мода":
                        category = EventCategory.FASHION
                    elif category_str == "гастрономия":
                        category = EventCategory.GASTRONOMY
                    elif category_str == "кино":
                        category = EventCategory.CINEMA
                    else:
                        category = EventCategory.OTHER
                except:
                    category = EventCategory.OTHER
                
                # Теги
                tags = []
                for tag_dict in event_dict.get('tags', []):
                    if isinstance(tag_dict, dict):
                        tags.append(TagModel(
                            name=tag_dict.get('name', ''),
                            slug=tag_dict.get('slug'),
                            id=tag_dict.get('id')
                        ))
                
                # Даты
                dates = []
                for date_dict in event_dict.get('dates', []):
                    if isinstance(date_dict, dict) and date_dict.get('start'):
                        dates.append(DateModel(
                            start=date_dict.get('start'),
                            end=date_dict.get('end'),
                            is_continuous=date_dict.get('is_continuous', False)
                        ))
                
                # Место
                place = None
                place_dict = event_dict.get('place')
                if place_dict and isinstance(place_dict, dict):
                    place = PlaceModel(
                        id=place_dict.get('id'),
                        title=place_dict.get('title', 'Неизвестно'),
                        address=place_dict.get('address'),
                        subway=place_dict.get('subway'),
                        coords=place_dict.get('coords'),
                        phone=place_dict.get('phone'),
                        site_url=place_dict.get('site_url'),
                        city=place_dict.get('city'),
                        is_closed=place_dict.get('is_closed', False),
                        working_hours=place_dict.get('working_hours')
                    )
                
                # Цена
                price = None
                price_dict = event_dict.get('price')
                if price_dict and isinstance(price_dict, dict):
                    price = PriceModel(
                        is_free=price_dict.get('is_free', False),
                        min=price_dict.get('min'),
                        max=price_dict.get('max'),
                        currency=price_dict.get('currency', 'RUB'),
                        description=price_dict.get('description')
                    )
                
                # Изображения
                images = []
                for img_dict in event_dict.get('images', []):
                    if isinstance(img_dict, dict):
                        images.append(ImageModel(
                            url=img_dict.get('url', ''),
                            thumbnail=img_dict.get('thumbnail'),
                            source=img_dict.get('source')
                        ))
                
                # Участники (упрощенно, так как в данных их может не быть)
                participants = []
                
                # Создаем Pydantic модель
                event_model = EventModel(
                    id=event_dict['id'],
                    title=event_dict['title'],
                    description=event_dict.get('description'),
                    category=category,
                    tags=tags,
                    dates=dates,
                    age_restriction=event_dict.get('age_restriction'),
                    place=place,
                    place_id=event_dict.get('place_id'),
                    price=price,
                    images=images,
                    participants=participants,
                    url=event_dict.get('url', ''),
                    favorites_count=event_dict.get('favorites_count', 0),
                    comments_count=event_dict.get('comments_count', 0),
                    location=event_dict.get('location', self.city),
                    is_free=event_dict.get('is_free', False)
                )
                
                pydantic_events.append(event_model)
                converted += 1
                
            except Exception as e:
                errors += 1
                logger.debug(f"Ошибка конвертации события {event_dict.get('id')}: {e}")
        
        print(f" Успешно: {converted}, ошибок: {errors}")
        return pydantic_events
    
    def save_to_json(self, events: List[Dict]) -> str:
        """Сохранение данных в JSON файл"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"events_{self.city}_{timestamp}.json"
        filepath = Path("data") / filename
        Path("data").mkdir(exist_ok=True)
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(events, f, ensure_ascii=False, indent=2)
        
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f" Файл сохранен: {filepath}")
        print(f" Размер: {size_mb:.2f} MB")
        
        return str(filepath)
    
    def show_statistics(self, events: List[Dict]):
        """Отображение статистики"""
        if not events:
            print(" Нет данных для статистики")
            return
        
        print(f" СТАТИСТИКА ДАННЫХ:")
        print(f"   Всего событий: {len(events)}")
        
        # По категориям
        categories = {}
        for event in events:
            cat = event.get('category', 'Неизвестно')
            categories[cat] = categories.get(cat, 0) + 1
        
        print(f"   Категорий: {len(categories)}")
        print(f"   Топ-5 категорий:")
        for cat, count in sorted(categories.items(), key=lambda x: x[1], reverse=True)[:5]:
            percentage = (count / len(events)) * 100
            print(f"     • {cat}: {count} ({percentage:.1f}%)")
        
        # Места
        events_with_place = sum(1 for e in events if e.get('place'))
        print(f"   Событий с указанием места: {events_with_place} ({events_with_place/len(events)*100:.1f}%)")
        
        # Бесплатные
        free_events = sum(1 for e in events if e.get('is_free'))
        print(f"   Бесплатных событий: {free_events} ({free_events/len(events)*100:.1f}%)")
        
        # Изображения
        events_with_images = sum(1 for e in events if e.get('images'))
        print(f"   Событий с изображениями: {events_with_images} ({events_with_images/len(events)*100:.1f}%)")
        
        # Даты
        events_with_dates = sum(1 for e in events if e.get('dates'))
        print(f"   Событий с датами: {events_with_dates} ({events_with_dates/len(events)*100:.1f}%)")
    
    def search_events(self, query: str, n_results: int = 10, category: str = None):
        """Поиск событий по запросу"""
        return self.vector_db.search_similar(query, n_results=n_results, city=self.city, category=category)
    
    def get_database_stats(self):
        """Получение статистики БД"""
        return self.vector_db.get_stats()

print(" Пайплайн создан!")

# ============ 6. ФУНКЦИИ ДЛЯ БЫСТРОГО ЗАПУСКА ============
async def quick_parse_moscow():
    """Быстрый парсинг Москвы"""
    print(" БЫСТРЫЙ ПАРСИНГ МОСКВЫ")
    print("="*50)
    
    pipeline = DataPipeline("msk")
    results = await pipeline.run_full_pipeline()
    
    if results:
        print(f"\n РЕЗУЛЬТАТЫ:")
        print(f"   Событий: {results['total_events']}")
        print(f"   Pydantic моделей: {results['pydantic_count']}")
        print(f"   JSON файл: {results['json_file'].split('/')[-1]}")
        print(f"   БД: {results['db_stats'].get('db_path', 'N/A')}")
    
    return results

async def quick_parse_spb():
    """Быстрый парсинг Санкт-Петербурга"""
    print(" БЫСТРЫЙ ПАРСИНГ САНКТ-ПЕТЕРБУРГА")
    print("="*50)
    
    pipeline = DataPipeline("spb")
    results = await pipeline.run_full_pipeline()
    
    if results:
        print(f"\n РЕЗУЛЬТАТЫ:")
        print(f"  Событий: {results['total_events']}")
        print(f"   Pydantic моделей: {results['pydantic_count']}")
        print(f"   JSON файл: {results['json_file'].split('/')[-1]}")
        print(f"   БД: {results['db_stats'].get('db_path', 'N/A')}")
    
    return results

async def parse_both_cities():
    """Парсинг обоих городов"""
    print("  ПАРСИНГ МОСКВЫ И САНКТ-ПЕТЕРБУРГА")
    print("="*50)
    
    results = {}
    
    # Москва
    print("\n1. МОСКВА")
    print("-"*40)
    results["msk"] = await quick_parse_moscow()
    
    # Пауза
    print("\n Ждем 2 секунды...")
    await asyncio.sleep(2)
    
    # Санкт-Петербург
    print("\n2. САНКТ-ПЕТЕРБУРГ")
    print("-"*40)
    results["spb"] = await quick_parse_spb()
    
    # Итоги
    print("\n" + "="*50)
    print(" ИТОГИ ПО ОБОИМ ГОРОДАМ")
    print("="*50)
    
    total_events = 0
    for city, result in results.items():
        if result:
            events = result.get('total_events', 0)
            total_events += events
            city_name = "Москва" if city == "msk" else "Санкт-Петербург"
            print(f"{city_name}: {events} событий")
    
    print(f"\n ВСЕГО: {total_events} событий")
    
    return results

# ============ 7. ЗАПУСК ============
print("\n" + "="*70)
print(" ПАРСЕР СОБЫТИЙ KUDAGO - SQLite ВЕКТОРНАЯ БАЗА")
print("="*70)
print("\n ВОЗМОЖНОСТИ:")
print("   15+ категорий событий")
print("   Парсинг Москвы и Санкт-Петербурга")
print("   Обогащение информации о местах")
print("   Pydantic модели для валидации")
print("   SQLite векторная база с эмбеддингами")
print("   Семантический поиск по событиям")
print("\n КОМАНДЫ ДЛЯ ЗАПУСКА:")
print("1. await quick_parse_moscow()    - быстрый парсинг Москвы")
print("2. await quick_parse_spb()       - быстрый парсинг СПб")
print("3. await parse_both_cities()     - парсинг обоих городов")
print("\n КОМАНДЫ ДЛЯ ПОИСКА ПОСЛЕ ПАРСИНГА:")
print('pipeline = DataPipeline("msk")')
print('results = pipeline.search_events("концерт рок", n_results=5)')
print('stats = pipeline.get_database_stats()')

 Библиотеки загружены и окружение настроено!
 Pydantic модели созданы!
 Векторная БД SQLite создана!
 Улучшенный парсер создан!
 Пайплайн создан!

 ПАРСЕР СОБЫТИЙ KUDAGO - SQLite ВЕКТОРНАЯ БАЗА

 ВОЗМОЖНОСТИ:
   15+ категорий событий
   Парсинг Москвы и Санкт-Петербурга
   Обогащение информации о местах
   Pydantic модели для валидации
   SQLite векторная база с эмбеддингами
   Семантический поиск по событиям

 КОМАНДЫ ДЛЯ ЗАПУСКА:
1. await quick_parse_moscow()    - быстрый парсинг Москвы
2. await quick_parse_spb()       - быстрый парсинг СПб
3. await parse_both_cities()     - парсинг обоих городов

 КОМАНДЫ ДЛЯ ПОИСКА ПОСЛЕ ПАРСИНГА:
pipeline = DataPipeline("msk")
results = pipeline.search_events("концерт рок", n_results=5)
stats = pipeline.get_database_stats()


In [2]:
await parse_both_cities()

  ПАРСИНГ МОСКВЫ И САНКТ-ПЕТЕРБУРГА

1. МОСКВА
----------------------------------------
 БЫСТРЫЙ ПАРСИНГ МОСКВЫ
 Векторная БД SQLite создана: events_msk_vector.db
 Размерность эмбеддингов: 384
 ЗАПУСК ПОЛНОГО ПАЙПЛАЙНА

1. ПАРСИНГ ДАННЫХ
----------------------------------------
 Начинаем парсинг города: Москва
 Категорий: 10
 Цель: 30 событий на категорию
--------------------------------------------------
   Категория: Концерты
   Категория: Театр
    Страница 1: 20 событий, спаршено 20
    Страница 1: 20 событий, спаршено 20
    Страница 2: 20 событий, спаршено 20
    Страница 2: 20 событий, спаршено 20
     Всего: 40 событий
   Категория: Выставки
     Всего: 40 событий
   Категория: Фестивали
    Страница 1: 20 событий, спаршено 20
    Страница 1: 20 событий, спаршено 20
    Страница 2: 20 событий, спаршено 20
    Страница 2: 20 событий, спаршено 20
     Всего: 40 событий
   Категория: Образование
     Всего: 40 событий
   Категория: Вечеринки
    Страница 1: 20 событий, спаршено 20

{'msk': {'total_events': 400,
  'pydantic_count': 82,
  'json_file': 'data\\events_msk_20251212_005956.json',
  'db_stats': {'total_events': 838,
   'total_embeddings': 1975,
   'cities_count': 1,
   'categories_count': 10,
   'events_by_city': {'msk': 838},
   'db_path': 'events_msk_vector.db'},
  'city': 'msk'},
 'spb': {'total_events': 320,
  'pydantic_count': 78,
  'json_file': 'data\\events_spb_20251212_010029.json',
  'db_stats': {'total_events': 784,
   'total_embeddings': 1258,
   'cities_count': 1,
   'categories_count': 10,
   'events_by_city': {'spb': 784},
   'db_path': 'events_spb_vector.db'},
  'city': 'spb'}}

In [ ]:
%%writefile vector_db_manager.py

import sqlite3
import numpy as np
import json
import os
import shutil
from datetime import datetime, timedelta  # ИСПРАВЛЕНО: добавлен timedelta
from typing import List, Dict, Optional, Any
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class VectorDBManager:
    """Менеджер для работы с векторной БД событий"""
    
    def __init__(self, db_path: str):
        """
        Инициализация менеджера базы данных
        
        Args:
            db_path: Путь к файлу базы данных SQLite
        """
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self.conn.row_factory = sqlite3.Row  # Для доступа к колонкам по имени
        logger.info(f"Подключено к БД: {db_path}")
    
    def get_database_stats(self) -> Dict[str, Any]:
        """Получение статистики базы данных"""
        cursor = self.conn.cursor()
        
        cursor.execute("SELECT COUNT(*) FROM events")
        total_events = cursor.fetchone()[0] or 0
        
        cursor.execute("SELECT COUNT(*) FROM event_embeddings")
        total_embeddings = cursor.fetchone()[0] or 0
        
        cursor.execute("SELECT COUNT(DISTINCT category) FROM events")
        categories_count = cursor.fetchone()[0] or 0
        
        cursor.execute("SELECT COUNT(DISTINCT city) FROM events")
        cities_count = cursor.fetchone()[0] or 0
        
        cursor.execute("SELECT category, COUNT(*) as count FROM events GROUP BY category ORDER BY count DESC")
        category_stats = cursor.fetchall()
        
        return {
            "total_events": total_events,
            "total_embeddings": total_embeddings,
            "categories_count": categories_count,
            "cities_count": cities_count,
            "category_stats": [dict(row) for row in category_stats],
            "db_path": self.db_path,
            "db_size_mb": os.path.getsize(self.db_path) / (1024 * 1024) if os.path.exists(self.db_path) else 0
        }
    
    def search_by_keyword(self, keyword: str, limit: int = 10) -> List[Dict]:
        """Поиск по ключевым словам (не векторный)"""
        cursor = self.conn.cursor()
        cursor.execute('''
            SELECT id, title, category, dates_text, price_text, place_text, url
            FROM events 
            WHERE title LIKE ? OR description LIKE ? OR embedding_text LIKE ?
            ORDER BY id DESC
            LIMIT ?
        ''', (f'%{keyword}%', f'%{keyword}%', f'%{keyword}%', limit))
        
        return [dict(row) for row in cursor.fetchall()]
    
    def search_by_category(self, category: str, limit: int = 20) -> List[Dict]:
        """Поиск событий по категории"""
        cursor = self.conn.cursor()
        cursor.execute('''
            SELECT id, title, category, dates_text, price_text, place_text, url, description
            FROM events 
            WHERE category = ?
            ORDER BY id DESC
            LIMIT ?
        ''', (category, limit))
        
        return [dict(row) for row in cursor.fetchall()]
    
    def get_recent_events(self, days: int = 7, limit: int = 20) -> List[Dict]:
        """Получение недавних событий"""
        cursor = self.conn.cursor()
        
        # Для упрощения, берем просто последние добавленные
        cursor.execute('''
            SELECT id, title, category, dates_text, price_text, place_text, url, parsed_at
            FROM events 
            ORDER BY parsed_at DESC
            LIMIT ?
        ''', (limit,))
        
        return [dict(row) for row in cursor.fetchall()]
    
    def export_to_csv(self, output_file: str = "events_export.csv") -> bool:
        """Экспорт событий в CSV"""
        try:
            import pandas as pd
            df = pd.read_sql_query("SELECT * FROM events", self.conn)
            df.to_csv(output_file, index=False, encoding='utf-8')
            logger.info(f"✅ Экспортировано {len(df)} событий в {output_file}")
            return True
        except ImportError:
            logger.error("❌ Для экспорта в CSV установите pandas: pip install pandas")
            return False
        except Exception as e:
            logger.error(f"❌ Ошибка экспорта в CSV: {e}")
            return False
    
    def export_simple_csv(self, output_file: str = "events_simple.csv"):
        """Экспорт в CSV без pandas (простой вариант)"""
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                SELECT id, title, category, dates_text, price_text, place_text, url, city, parsed_at
                FROM events
            ''')
            
            with open(output_file, 'w', encoding='utf-8') as f:
                # Заголовок
                f.write("ID;Название;Категория;Даты;Цена;Место;URL;Город;Дата парсинга\n")
                
                # Данные
                for row in cursor.fetchall():
                    # Экранируем кавычки и заменяем разделители
                    title = str(row[1]).replace(';', ',').replace('"', "'")
                    category = str(row[2]).replace(';', ',')
                    dates = str(row[3]).replace(';', ',').replace('"', "'") if row[3] else ""
                    price = str(row[4]).replace(';', ',') if row[4] else ""
                    place = str(row[5]).replace(';', ',').replace('"', "'") if row[5] else ""
                    url = str(row[6]) if row[6] else ""
                    city = str(row[7]) if row[7] else ""
                    parsed_at = str(row[8]) if row[8] else ""
                    
                    f.write(f'{row[0]};"{title}";{category};"{dates}";{price};"{place}";{url};{city};{parsed_at}\n')
            
            logger.info(f"✅ Экспортировано в {output_file}")
            return True
            
        except Exception as e:
            logger.error(f"❌ Ошибка экспорта: {e}")
            return False
    
    def export_to_json(self, output_file: str = "events_export.json", limit: int = 100):
        """Экспорт событий в JSON"""
        try:
            cursor = self.conn.cursor()
            cursor.execute(f'''
                SELECT e.*, ed.dates_json, ed.price_json, ed.place_json, 
                       ed.tags_json, ed.images_json, ed.participants_json
                FROM events e
                LEFT JOIN event_details ed ON e.id = ed.event_id
                ORDER BY e.id DESC
                LIMIT ?
            ''', (limit,))
            
            columns = [desc[0] for desc in cursor.description]
            events = []
            
            for row in cursor.fetchall():
                event_dict = {}
                for i, col in enumerate(columns):
                    value = row[i]
                    # Пытаемся парсить JSON поля
                    if col.endswith('_json') and value:
                        try:
                            event_dict[col] = json.loads(value)
                        except:
                            event_dict[col] = value
                    else:
                        event_dict[col] = value
                events.append(event_dict)
            
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(events, f, ensure_ascii=False, indent=2)
            
            logger.info(f"✅ Экспортировано {len(events)} событий в {output_file}")
            return True
            
        except Exception as e:
            logger.error(f"❌ Ошибка экспорта в JSON: {e}")
            return False
    
    def cleanup_old_events(self, days_old: int = 30) -> int:
        """Очистка старых событий"""
        try:
            cursor = self.conn.cursor()
            cutoff_date = (datetime.now() - timedelta(days=days_old)).isoformat()
            
            # Сначала удаляем связанные записи
            cursor.execute("DELETE FROM event_embeddings WHERE event_id IN (SELECT id FROM events WHERE parsed_at < ?)", (cutoff_date,))
            cursor.execute("DELETE FROM event_details WHERE event_id IN (SELECT id FROM events WHERE parsed_at < ?)", (cutoff_date,))
            
            # Затем удаляем сами события
            cursor.execute("DELETE FROM events WHERE parsed_at < ?", (cutoff_date,))
            deleted = cursor.rowcount
            
            self.conn.commit()
            logger.info(f"🧹 Удалено {deleted} старых событий (старше {days_old} дней)")
            return deleted
            
        except Exception as e:
            logger.error(f"❌ Ошибка очистки старых событий: {e}")
            self.conn.rollback()
            return 0
    
    def backup_database(self, backup_dir: str = "backups") -> Optional[str]:
        """Создание резервной копии базы данных"""
        try:
            os.makedirs(backup_dir, exist_ok=True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            db_name = os.path.basename(self.db_path)
            backup_file = os.path.join(backup_dir, f"{db_name}_{timestamp}")
            
            # Копируем файл БД
            shutil.copy2(self.db_path, backup_file)
            
            logger.info(f"💾 Создана резервная копия: {backup_file}")
            return backup_file
            
        except Exception as e:
            logger.error(f"❌ Ошибка создания резервной копии: {e}")
            return None
    
    def get_event_by_id(self, event_id: int) -> Optional[Dict]:
        """Получение полной информации о событии по ID"""
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                SELECT e.*, ed.dates_json, ed.price_json, ed.place_json, 
                       ed.tags_json, ed.images_json, ed.participants_json
                FROM events e
                LEFT JOIN event_details ed ON e.id = ed.event_id
                WHERE e.id = ?
            ''', (event_id,))
            
            row = cursor.fetchone()
            if not row:
                return None
            
            # Преобразуем в словарь
            columns = [desc[0] for desc in cursor.description]
            event_dict = {}
            
            for i, col in enumerate(columns):
                value = row[i]
                if col.endswith('_json') and value:
                    try:
                        event_dict[col] = json.loads(value)
                    except:
                        event_dict[col] = value
                else:
                    event_dict[col] = value
            
            return event_dict
            
        except Exception as e:
            logger.error(f"❌ Ошибка получения события {event_id}: {e}")
            return None
    
    def print_statistics(self):
        """Вывод статистики в консоль"""
        stats = self.get_database_stats()
        
        print("="*60)
        print("📊 СТАТИСТИКА БАЗЫ ДАННЫХ СОБЫТИЙ")
        print("="*60)
        print(f"📁 Файл БД: {stats['db_path']}")
        print(f"📦 Размер: {stats['db_size_mb']:.2f} MB")
        print(f"🎫 Всего событий: {stats['total_events']}")
        print(f"🔢 Эмбеддингов: {stats['total_embeddings']}")
        print(f"🏙️ Городов: {stats['cities_count']}")
        print(f"🏷️ Категорий: {stats['categories_count']}")
        
        if stats['category_stats']:
            print("\n🏆 Топ категорий:")
            for cat_stat in stats['category_stats'][:10]:
                print(f"  {cat_stat['category']}: {cat_stat['count']} событий")
    
    def close(self):
        """Закрытие соединения с БД"""
        if self.conn:
            self.conn.close()
            logger.info("Соединение с БД закрыто")
    
    def __enter__(self):
        """Поддержка контекстного менеджера (with statement)"""
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Автоматическое закрытие соединения при выходе из контекста"""
        self.close()


# ============ ФУНКЦИИ ДЛЯ БЫСТРОГО ИСПОЛЬЗОВАНИЯ ============

def quick_analyze(db_path: str = "events_msk_vector.db"):
    """Быстрый анализ базы данных"""
    with VectorDBManager(db_path) as manager:
        manager.print_statistics()
        
        print("\n🔍 Примеры событий:")
        recent = manager.get_recent_events(limit=3)
        for event in recent:
            print(f"  • {event['title'][:50]}... ({event['category']})")

def export_all_formats(db_path: str = "events_msk_vector.db"):
    """Экспорт во все форматы"""
    with VectorDBManager(db_path) as manager:
        # Экспорт в CSV
        manager.export_simple_csv("events_export.csv")
        
        # Экспорт в JSON
        manager.export_to_json("events_export.json", limit=50)
        
        # Статистика
        manager.print_statistics()

def search_and_export(keyword: str, db_path: str = "events_msk_vector.db"):
    """Поиск и экспорт результатов"""
    with VectorDBManager(db_path) as manager:
        print(f"🔍 Поиск: '{keyword}'")
        results = manager.search_by_keyword(keyword, limit=20)
        
        print(f"📄 Найдено: {len(results)} событий")
        
        # Сохраняем результаты
        if results:
            with open(f"search_{keyword}.json", 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"💾 Результаты сохранены в search_{keyword}.json")
            
            # Выводим первые 5
            for i, event in enumerate(results[:5], 1):
                print(f"{i}. {event['title']}")
                print(f"   Категория: {event['category']}")
                if event.get('place_text'):
                    print(f"   Место: {event['place_text'][:50]}...")
                print()


if __name__ == "__main__":
    # Пример использования при запуске файла напрямую
    print("🚀 Запуск менеджера векторной БД...")
    
    # Автоматический анализ, если файл БД существует
    if os.path.exists("events_msk_vector.db"):
        quick_analyze("events_msk_vector.db")
    else:
        print("❌ Файл БД не найден. Сначала запустите парсер.")